In [419]:
import pandas as pd
import random 
import numpy as np
from prettytable import PrettyTable

random.seed(1212456)

# Parameters
POP_SIZE = 10
NUM_GENERATIONS = 100
MUTATION_RATE = 0.1
PERFECT_FITNESS = 0

In [420]:

# Professors Data
professors = pd.DataFrame({
    'ProfessorID': [1, 2, 3, 4, 5],
    'Name': ['Kashif Munir', 'Mukhtaar Ullah', 'Naveed Ahmed', 'Farukh Bashir', 'Amina Ashfaq'],
    'Enc': ["000","001","010","011","100"]
})

# Courses Data updated with ProfessorID
courses = pd.DataFrame({
    'CourseID': [1, 2, 3, 4, 5, 6],
    'Name': ['OS', 'OS Lab', 'AI', 'AI Lab', 'CNET', 'Network Security'],
    'IsLab': [False, True, False, True, False, False],
    'ProfessorID': [1, 1, 3, 3, 5, 5],
    'Enc': ["000","001","010","011","100", "101"]
})

# Sections Data
sections = pd.DataFrame({
    'SectionID': [1, 1, 2, 3, 4, 5, 5],
    'CourseID': [1, 6, 2, 3, 4, 5, 1],
    'Capacity': [50, 50, 25, 55, 20, 60, 60],  # Students count fits the room capacities
    'Enc': ["000","001","010","011","100","101","110"]
})


# Rooms Data
rooms = pd.DataFrame({
    'RoomID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'Capacity': [60, 120, 60, 60, 60, 120, 60, 120, 60, 60, 60, 120, 60, 60, 120],
    'Floor': [1, 1, 2, 2, 3, 3, 0, 0, 1, 1, 0, 0, 1, 1, 2],
    'Enc': ["000", "001", "010", "011", "100", "101", "110", "111", "000", "001", "010", "011", "100", "101", "110"]
})

# Time Slots Data
timeslots = pd.DataFrame({
    'TimeSlotID': [1, 2, 3, 4, 5, 6],
    'Timings' : [(8.30,9.50),(10,11.20),(11.30,12.50),(13,14.20),(14.30,15.50),(16,17.20)],
    'Enc': ["000", "001", "010", "011", "100", "101"]
})

#Days Data
days= pd.DataFrame({
    "Day": ["Monday","Tuesday","Wednesday","Thursday","Friday"],
    'Enc': ["000","001","010","011","100"]
})


print("Professors:")
print(professors)
print("\nCourses:")
print(courses)
print("\nRooms:")
print(rooms)
print("\nSections:")
print(sections)
print("\nTimeSlots:")
print(timeslots)
print("\nDays:")
print(days)


Professors:
   ProfessorID            Name  Enc
0            1    Kashif Munir  000
1            2  Mukhtaar Ullah  001
2            3    Naveed Ahmed  010
3            4   Farukh Bashir  011
4            5    Amina Ashfaq  100

Courses:
   CourseID              Name  IsLab  ProfessorID  Enc
0         1                OS  False            1  000
1         2            OS Lab   True            1  001
2         3                AI  False            3  010
3         4            AI Lab   True            3  011
4         5              CNET  False            5  100
5         6  Network Security  False            5  101

Rooms:
    RoomID  Capacity  Floor  Enc
0        1        60      1  000
1        2       120      1  001
2        3        60      2  010
3        4        60      2  011
4        5        60      3  100
5        6       120      3  101
6        7        60      0  110
7        8       120      0  111
8        9        60      1  000
9       10        60      1  001
10    

In [421]:
def generate_chromosome(courses, sections, professors, rooms, timeslots, days):
    detailed_chromosome = []
    encoded_chromosome = []
    for section_id in sections['SectionID'].unique():
        section_courses = sections[sections['SectionID'] == section_id]
        
        for idx, row in section_courses.iterrows():
            course = courses[courses['CourseID'] == row['CourseID']].iloc[0]
            professor = professors[professors['ProfessorID'] == course['ProfessorID']].iloc[0]
            room = rooms[rooms['Capacity'] >= row['Capacity']].sample().iloc[0]
            first_day = days.sample().iloc[0]
            first_timeslot_index = random.randint(0, len(timeslots) - 2)  # Ensure there's enough space for a lab
            first_timeslot = timeslots.iloc[first_timeslot_index]

            # Convert timeslot tuple to string format for detailed chromosome
            first_timeslot_str = f"{first_timeslot['Timings'][0]:.2f},{first_timeslot['Timings'][1]:.2f}"

            # Detailed gene construction
            detailed_gene = {
                'Course': course['Name'],
                'Type': 'Lab' if course['IsLab'] else 'Theory',
                'Section': row['SectionID'],
                'Capacity': row['Capacity'],
                'Professor': professor['Name'],
                'First Lecture Day': first_day['Day'],
                'First Lecture Timeslot': first_timeslot_str,
                'First Lecture Room': room['RoomID'],
                'First Lecture Room Capacity': room['Capacity']
            }

            # Encoded gene construction using binary strings
            encoded_gene = {
                'Course': course['Enc'],
                'Type': '1' if course['IsLab'] else '0',
                'Section': row['Enc'],
                'Capacity': format(row['Capacity'], '07b'),
                'Professor': professor['Enc'],
                'First Lecture Day': first_day['Enc'],
                'First Lecture Timeslot': first_timeslot['Enc'],
                'First Lecture Room': room['Enc'],
                'First Lecture Room Capacity': '1' if room['Capacity'] == 120 else '0'
            }

            if course['IsLab']:
                # For lab courses, ensure two consecutive time slots
                second_timeslot = timeslots.iloc[first_timeslot_index + 1]
                second_timeslot_str = f"{second_timeslot['Timings'][0]:.2f},{second_timeslot['Timings'][1]:.2f}"

                detailed_gene.update({
                    'Second Lecture Day': first_day['Day'],
                    'Second Lecture Timeslot': second_timeslot_str,
                    'Second Lecture Room': room['RoomID'],
                    'Second Lecture Room Capacity': room['Capacity']
                })

                encoded_gene.update({
                    'Second Lecture Day': first_day['Enc'],
                    'Second Lecture Timeslot': second_timeslot['Enc'],
                    'Second Lecture Room': room['Enc'],
                    'Second Lecture Room Capacity': '1' if room['Capacity'] == 120 else '0'
                })

            else:
                # Ensure second lecture day is not the same or adjacent to the first
                valid_days = days[~days['Day'].isin([first_day['Day'], day_before(first_day['Day'], days), day_after(first_day['Day'], days)])]
                second_day = valid_days.sample().iloc[0]
                second_timeslot = timeslots.sample().iloc[0]
                second_room = rooms[rooms['Capacity'] >= row['Capacity']].sample().iloc[0]

                second_timeslot_str = f"{second_timeslot['Timings'][0]:.2f},{second_timeslot['Timings'][1]:.2f}"

                detailed_gene.update({
                    'Second Lecture Day': second_day['Day'],
                    'Second Lecture Timeslot': second_timeslot_str,
                    'Second Lecture Room': second_room['RoomID'],
                    'Second Lecture Room Capacity': second_room['Capacity']
                })

                encoded_gene.update({
                    'Second Lecture Day': second_day['Enc'],
                    'Second Lecture Timeslot': second_timeslot['Enc'],
                    'Second Lecture Room': second_room['Enc'],
                    'Second Lecture Room Capacity': '1' if second_room['Capacity'] == 120 else '0'
                })

            # Append the detailed and encoded genes to respective chromosomes
            detailed_chromosome.append(detailed_gene)
            encoded_chromosome.append(encoded_gene)
    
    return detailed_chromosome, encoded_chromosome

def day_before(day, days_df):
    # Calculate the index of the previous day in the dataframe
    index = days_df.index[days_df['Day'] == day].tolist()[0] - 1
    if index < 0:
        return None  # No previous day
    return days_df.iloc[index]['Day']

def day_after(day, days_df):
    # Calculate the index of the next day in the dataframe
    index = days_df.index[days_df['Day'] == day].tolist()[0] + 1
    if index >= len(days_df):
        return None  # No next day
    return days_df.iloc[index]['Day']

# Example usage:
detailed_chromosome, encoded_chromosome = generate_chromosome(courses, sections, professors, rooms, timeslots, days)
#print("Detailed Chromosome:", detailed_chromosome)
#print("Encoded Chromosome:", encoded_chromosome)


# Generate a single chromosome with both detailed and encoded data
detailed_chromosome, encoded_chromosome = generate_chromosome(courses, sections, professors, rooms, timeslots, days)
#print("Generated Detailed Chromosome:", detailed_chromosome)
#print("Generated Encoded Chromosome:", encoded_chromosome)

df = pd.DataFrame(detailed_chromosome)
print(df)
#print(encoded_chromosome)

             Course    Type  Section  Capacity     Professor  \
0                OS  Theory        1        50  Kashif Munir   
1  Network Security  Theory        1        50  Amina Ashfaq   
2            OS Lab     Lab        2        25  Kashif Munir   
3                AI  Theory        3        55  Naveed Ahmed   
4            AI Lab     Lab        4        20  Naveed Ahmed   
5              CNET  Theory        5        60  Amina Ashfaq   
6                OS  Theory        5        60  Kashif Munir   

  First Lecture Day First Lecture Timeslot  First Lecture Room  \
0          Thursday            10.00,11.20                   5   
1         Wednesday              8.30,9.50                   3   
2         Wednesday            13.00,14.20                  13   
3           Tuesday            11.30,12.50                  14   
4           Tuesday            13.00,14.20                   5   
5          Thursday            13.00,14.20                  13   
6         Wednesday      

In [422]:
# def generate_population(courses, sections, professors, rooms, timeslots, days):
#     population = []
#     for _ in range(POP_SIZE):
#         detailed_chromosome, encoded_chromosome = generate_chromosome(courses, sections, professors, rooms, timeslots, days)
#         population.append((detailed_chromosome, encoded_chromosome))
#     return population


# population = generate_population(courses, sections, professors, rooms, timeslots, days)

In [423]:
# for i in range(len(population)):
#     for j in range (2):
#         df = pd.DataFrame(population[i][j])  # Detailed walay k liyay 0, chnge to 1 for encoded bruj
#         print(df)

In [431]:
def calculate_fitness(detailed_chromosome, encoded_chromosome, timeslots, rooms, days):
    penalty = 0
    
    # Construct a dictionary from timeslot Enc to Timings for quick lookup
    timeslot_dict = {ts['Enc']: ts['Timings'] for idx, ts in timeslots.iterrows()}

      # Iterate over pairs of genes in the encoded chromosome to check for conflicts
    for i in range(len(encoded_chromosome)):
        gene1 = encoded_chromosome[i]
        prof11, day11, slot11, room11 = gene1['Professor'], gene1['First Lecture Day'], gene1['First Lecture Timeslot'], gene1['First Lecture Room']
        day12, slot12, room12 = gene1['Second Lecture Day'], gene1['Second Lecture Timeslot'], gene1['Second Lecture Room']

        for j in range(i + 1, len(encoded_chromosome)):
            gene2 = encoded_chromosome[j]
            prof21, day21, slot21, room21 = gene2['Professor'], gene2['First Lecture Day'], gene2['First Lecture Timeslot'], gene2['First Lecture Room']
            day22, slot22, room22 = gene2['Second Lecture Day'], gene2['Second Lecture Timeslot'], gene2['Second Lecture Room']
            
            # Check clashes between all combinations of lectures
            if prof11 == prof21:
                # First Lecture of Gene1 with First and Second of Gene2
                if (day11 == day21 and timeslot_overlap(timeslot_dict[slot11], timeslot_dict[slot21])) or \
                   (day11 == day22 and timeslot_overlap(timeslot_dict[slot11], timeslot_dict[slot22])):
                    penalty += 10
                # Second Lecture of Gene1 with First and Second of Gene2
                if (day12 == day21 and timeslot_overlap(timeslot_dict[slot12], timeslot_dict[slot21])) or \
                   (day12 == day22 and timeslot_overlap(timeslot_dict[slot12], timeslot_dict[slot22])):
                    penalty += 10

            # Check room scheduling conflicts
            if (room11 == room21 and day11 == day21 and timeslot_overlap(timeslot_dict[slot11], timeslot_dict[slot21])) or \
               (room11 == room22 and day11 == day22 and timeslot_overlap(timeslot_dict[slot11], timeslot_dict[slot22])):
                penalty += 10
            if (room12 == room21 and day12 == day21 and timeslot_overlap(timeslot_dict[slot12], timeslot_dict[slot21])) or \
               (room12 == room22 and day12 == day22 and timeslot_overlap(timeslot_dict[slot12], timeslot_dict[slot22])):
                penalty += 10

            # Soft constraints for both lectures in gene1
            if detailed_chromosome[i]['Type'] == 'Theory':
                if not in_morning(timeslot_dict[slot11]):
                    penalty += 1
                if 'Second Lecture Timeslot' in gene1 and not in_morning(timeslot_dict[slot12]):
                    penalty += 1
            if detailed_chromosome[i]['Type'] == 'Lab':
                if not in_afternoon(timeslot_dict[slot11]):
                    penalty += 1
                if 'Second Lecture Timeslot' in gene1 and not in_afternoon(timeslot_dict[slot12]):
                    penalty += 1


    return -penalty

# Helper functions for time checks
def timeslot_overlap(timeslot1, timeslot2):
    start1, end1 = timeslot1
    start2, end2 = timeslot2
    return max(start1, start2) < min(end1, end2)

def in_morning(timeslot):
    start, _ = timeslot
    return start < 12.50  # Assuming 12:50 as the end of morning session

def in_afternoon(timeslot):
    start, _ = timeslot
    return start >= 13  # Assuming 13:00 as the start of afternoon session

# Example usage
detailed_chromosome, encoded_chromosome = generate_chromosome(courses, sections, professors, rooms, timeslots, days)
fitness_score = calculate_fitness(detailed_chromosome, encoded_chromosome, timeslots, rooms, days)
print("Fitness Score:", fitness_score)

# Display the detailed chromosome in a DataFrame for better readability
df = pd.DataFrame(detailed_chromosome)
print(df)



Fitness Score: -30
             Course    Type  Section  Capacity     Professor  \
0                OS  Theory        1        50  Kashif Munir   
1  Network Security  Theory        1        50  Amina Ashfaq   
2            OS Lab     Lab        2        25  Kashif Munir   
3                AI  Theory        3        55  Naveed Ahmed   
4            AI Lab     Lab        4        20  Naveed Ahmed   
5              CNET  Theory        5        60  Amina Ashfaq   
6                OS  Theory        5        60  Kashif Munir   

  First Lecture Day First Lecture Timeslot  First Lecture Room  \
0          Thursday            13.00,14.20                  12   
1           Tuesday            11.30,12.50                  13   
2         Wednesday              8.30,9.50                  10   
3         Wednesday            10.00,11.20                  12   
4            Friday            13.00,14.20                   7   
5           Tuesday            10.00,11.20                   7   
6     

In [429]:

def generate_population(pop_size, courses, sections, professors, rooms, timeslots, days):
    return [generate_chromosome(courses, sections, professors, rooms, timeslots, days) for _ in range(pop_size)]

{
# def regenerate_detailed_from_encoded(encoded_chromosome, courses, professors, rooms, timeslots, days):
#     detailed_chromosome = []
#     for gene in encoded_chromosome:
#         course = courses.loc[courses['Enc'] == gene['Course']].iloc[0]
#         professor = professors.loc[professors['Enc'] == gene['Professor']].iloc[0]
#         room = rooms.loc[rooms['Enc'] == gene['First Lecture Room']].iloc[0]
#         timeslot = timeslots.loc[timeslots['Enc'] == gene['First Lecture Timeslot']].iloc[0]
#         day = days.loc[days['Enc'] == gene['First Lecture Day']].iloc[0]

#         # First lecture details
#         detailed_gene = {
#             'Course': course['Name'],
#             'Type': 'Lab' if course['IsLab'] else 'Theory',
#             'Section': course['CourseID'],
#             'Section Capacity': room['Capacity'],
#             'Professor': professor['Name'],
#             'First Lecture Day': day['Day'],
#             'First Lecture Timeslot': timeslot['Timings'],
#             'First Lecture Room': room['RoomID'],
#             'First Lecture Room Capacity': room['Capacity']
#         }

#         # Second lecture details, if they exist
#         if 'Second Lecture Day' in gene:
#             second_day = days.loc[days['Enc'] == gene['Second Lecture Day']].iloc[0]
#             second_timeslot = timeslots.loc[timeslots['Enc'] == gene['Second Lecture Timeslot']].iloc[0]
#             second_room = rooms.loc[rooms['Enc'] == gene['Second Lecture Room']].iloc[0]

#             detailed_gene.update({
#                 'Second Lecture Day': second_day['Day'],
#                 'Second Lecture Timeslot': second_timeslot['Timings'],
#                 'Second Lecture Room': second_room['RoomID'],
#                 'Second Lecture Room Capacity': second_room['Capacity']
#             })

#         detailed_chromosome.append(detailed_gene)

#     return detailed_chromosome
}
def regenerate_detailed_from_encoded(encoded_chromosome, courses, professors, rooms, timeslots, days):
    detailed_chromosome = []
    updated_encoded_chromosome = []
    for gene in encoded_chromosome:
        # Fetch information based on encoded values
        course = courses.loc[courses['Enc'] == gene['Course']].iloc[0]
        professor = professors.loc[professors['Enc'] == gene['Professor']].iloc[0]
        room = rooms.loc[rooms['Enc'] == gene['First Lecture Room']].iloc[0]
        first_timeslot = timeslots.loc[timeslots['Enc'] == gene['First Lecture Timeslot']].iloc[0]
        first_day = days.loc[days['Enc'] == gene['First Lecture Day']].iloc[0]
        section = sections.loc[sections['Enc'] == gene['Section']].iloc[0]

        # Setup the first lecture details
        detailed_gene = {
            'Course': course['Name'],
            'Type': 'Lab' if course['IsLab'] else 'Theory',
            'Section': section['SectionID'],
            'Section Capacity': section['Capacity'],
            'Professor': professor['Name'],
            'First Lecture Day': first_day['Day'],
            'First Lecture Timeslot': f"{first_timeslot['Timings'][0]:.2f}-{first_timeslot['Timings'][1]:.2f}",
            'First Lecture Room': room['RoomID'],
            'First Lecture Room Capacity': room['Capacity']
        }

        # Update encoded_gene
        updated_gene = gene.copy()
        if course['IsLab']:
            # Ensure two consecutive slots
            second_timeslot_index = (timeslots.index[timeslots['Enc'] == gene['First Lecture Timeslot']][0] + 1) % len(timeslots)
            second_timeslot = timeslots.iloc[second_timeslot_index]
            updated_gene['Second Lecture Timeslot'] = second_timeslot['Enc']

            detailed_gene.update({
                'Second Lecture Day': first_day['Day'],
                'Second Lecture Timeslot': f"{second_timeslot['Timings'][0]:.2f}-{second_timeslot['Timings'][1]:.2f}",
                'Second Lecture Room': room['RoomID'],
                'Second Lecture Room Capacity': room['Capacity']
            })
        else:
            # Theory courses should have lectures on non-consecutive and non-adjacent days
            adjacent_days = get_adjacent_days(first_day['Day'], days)
            available_days = days.loc[~days['Day'].isin(adjacent_days + [first_day['Day']])]
            second_day = available_days.sample().iloc[0]
            second_timeslot = timeslots.sample().iloc[0]
            second_room = rooms.loc[rooms['Capacity'] >= section['Capacity']].sample().iloc[0]

            updated_gene.update({
                'Second Lecture Day': second_day['Enc'],
                'Second Lecture Timeslot': second_timeslot['Enc'],
                'Second Lecture Room': second_room['Enc']
            })

            detailed_gene.update({
                'Second Lecture Day': second_day['Day'],
                'Second Lecture Timeslot': f"{second_timeslot['Timings'][0]:.2f}-{second_timeslot['Timings'][1]:.2f}",
                'Second Lecture Room': second_room['RoomID'],
                'Second Lecture Room Capacity': second_room['Capacity']
            })

        detailed_chromosome.append(detailed_gene)
        updated_encoded_chromosome.append(updated_gene)

    return detailed_chromosome, updated_encoded_chromosome

def get_adjacent_days(current_day, days_df):
    """Utility function to get the adjacent days to prevent scheduling on consecutive days for theory courses."""
    current_index = days_df.index[days_df['Day'] == current_day].tolist()[0]
    adjacent_indices = [current_index - 1, current_index + 1]
    adjacent_days = days_df.loc[days_df.index.isin(adjacent_indices), 'Day'].tolist()
    return adjacent_days



def select(population, fitness_scores, num_parents):
    # Tournament selection
    selected_indices = sorted(range(len(fitness_scores)), key=lambda i: fitness_scores[i], reverse=True)[:num_parents]
    return [population[i] for i in selected_indices]

{
# def crossover(parent1, parent2):
#     # One-point crossover
#     crossover_index = random.randint(1, len(parent1) - 1)
#     child1 = parent1[:crossover_index] + parent2[crossover_index:]
#     child2 = parent2[:crossover_index] + parent1[crossover_index:]
#     return child1, child2

# def mutate(encoded_chromosome, mutation_rate, timeslots):
#     for i in range(len(encoded_chromosome)):
#         if random.random() < mutation_rate:
#             # Perform mutation (e.g., change a timeslot or room)
#             gene = encoded_chromosome[i]
            
#             # Mutate the timeslot by selecting a new one randomly from the timeslot encodings
#             new_timeslot_enc = random.choice(timeslots['Enc'].tolist())
#             gene['First Lecture Timeslot'] = new_timeslot_enc
            
#             # Optionally mutate other attributes such as room
#             if random.random() < 0.5:  # Additional random condition for room mutation
#                 new_room_enc = random.choice(rooms['Enc'].tolist())
#                 gene['First Lecture Room'] = new_room_enc
            
#             encoded_chromosome[i] = gene

#     return encoded_chromosome
}

def crossover(parent1, parent2):
    # One-point crossover for the encoded parts
    crossover_index = random.randint(1, len(parent1[1]) - 1)
    child1_encoded = parent1[1][:crossover_index] + parent2[1][crossover_index:]
    child2_encoded = parent2[1][:crossover_index] + parent1[1][crossover_index:]
    
    # Regenerate the detailed parts from the new encoded parts
    child1_detailed,child1_encoded = regenerate_detailed_from_encoded(child1_encoded, courses, professors, rooms, timeslots, days)
    child2_detailed,child2_encoded = regenerate_detailed_from_encoded(child2_encoded, courses, professors, rooms, timeslots, days)
    
    return (child1_detailed, child1_encoded), (child2_detailed, child2_encoded)

def mutate(child, mutation_rate, timeslots):
    encoded_chromosome = child[1]
    for i in range(len(encoded_chromosome)):
        if random.random() < mutation_rate:
            # Mutate the timeslot
            new_timeslot_enc = random.choice(timeslots['Enc'].tolist())
            encoded_chromosome[i]['First Lecture Timeslot'] = new_timeslot_enc
            
            # Optional: Mutate the room
            if random.random() < 0.5:
                new_room_enc = random.choice(rooms['Enc'].tolist())
                encoded_chromosome[i]['First Lecture Room'] = new_room_enc

    # Regenerate detailed part after mutation
    detailed_chromosome,encoded_chromosome = regenerate_detailed_from_encoded(encoded_chromosome, courses, professors, rooms, timeslots, days)
    return detailed_chromosome, encoded_chromosome
{
# def genetic_algorithm(courses, sections, professors, rooms, timeslots, days, pop_size, num_generations, mutation_rate):
#     population = generate_population(pop_size, courses, sections, professors, rooms, timeslots, days)
#     best_fitness = float('-inf')
#     best_solution = None

#     for generation in range(num_generations):
#         # Separate detailed and encoded chromosomes for easier handling
#         detailed_population, encoded_population = zip(*population)
#         fitness_scores = [calculate_fitness(detailed, encoded, timeslots, rooms, days) for detailed, encoded in population]
        
#         if max(fitness_scores) > best_fitness:
#             best_fitness = max(fitness_scores)
#             best_index = fitness_scores.index(best_fitness)
#             best_solution = (detailed_population[best_index], encoded_population[best_index])

#         print(f'Generation {generation}: Best Fitness {best_fitness}')

#         # Check for perfect fitness
#         if best_fitness >= PERFECT_FITNESS:
#             print("Perfect fitness reached. Exiting the loop.")
#             best_solution = (detailed_population[best_index], encoded_population[best_index])
#             break
        
#        # Selection
#         num_parents = pop_size // 2
#         if num_parents % 2 != 0:  # Ensure we always have an even number of parents
#             num_parents -= 1
#         parents = select(population, fitness_scores, num_parents)


#         # Crossover to generate children
#         children = []
#         for i in range(0, len(parents), 2):
#             parent1, parent2 = parents[i], parents[i+1]
#             child1, child2 = crossover(parent1[1], parent2[1])  # Crossover encoded parts
#             children.append(child1)
#             children.append(child2)

#         # Mutation applied to children
#         mutated_children = [mutate(child, mutation_rate, timeslots) for child in children]

#         # Combine parents and mutated children to form the new population
#         population = parents + list(zip(detailed_population[:len(mutated_children)], mutated_children))

#     return best_solution
 }

def genetic_algorithm(courses, sections, professors, rooms, timeslots, days, pop_size, num_generations, mutation_rate):
    population = generate_population(pop_size, courses, sections, professors, rooms, timeslots, days)
    best_fitness = float('-inf')
    best_solution = None

    for generation in range(num_generations):
        # Evaluate fitness
        #Separate detailed and encoded chromosomes for easier handling
        detailed_population, encoded_population = zip(*population)
        fitness_scores = [calculate_fitness(detailed, encoded, timeslots, rooms, days) for detailed, encoded in population]

        # Find the best solution in the current generation
        if max(fitness_scores) > best_fitness:
            best_fitness = max(fitness_scores)
            best_index = fitness_scores.index(best_fitness)
            best_solution = (detailed_population[best_index], encoded_population[best_index])

        print(f'Generation {generation}: Best Fitness {best_fitness}')

        # Check for perfect fitness
        if best_fitness >= PERFECT_FITNESS:
            print("Perfect fitness reached. Exiting the loop.")
            best_solution = (detailed_population[best_index], encoded_population[best_index])
            break

        # Selection and reproduction
        num_parents = pop_size // 2
        if num_parents % 2 != 0:  # Ensure we always have an even number of parents
            num_parents -= 1
        parents = select(population, fitness_scores, num_parents)

        # Crossover and mutation
        children = []
        for i in range(0, len(parents) - 1, 2):
            child1, child2 = crossover(parents[i], parents[i+1])
            mutated_child1 = mutate(child1, mutation_rate, timeslots)
            mutated_child2 = mutate(child2, mutation_rate, timeslots)
            children.extend([mutated_child1, mutated_child2])

        # Create the new generation
        population = parents + children

    return best_solution

# Run the genetic algorithm
best_chromosome = genetic_algorithm(courses, sections, professors, rooms, timeslots, days, POP_SIZE, NUM_GENERATIONS, MUTATION_RATE)

df=pd.DataFrame(best_chromosome[0])

print(df)

# Create a PrettyTable instance
table = PrettyTable()

# Add column names to the table
table.field_names = df.columns

print("Best Solution Found:\n")
# Add rows to the table from the DataFrame
for row in df.values:
    table.add_row(row)

# Print the table
#print(table)

#|  Added this cuz gay print statement. 
#v

# Define the file path to store the table
output_file_path = "output_table.txt"

# Write the table to the text file
with open(output_file_path, "w") as file:
    file.write(str(table))



Generation 0: Best Fitness -24
Generation 1: Best Fitness -24
Generation 2: Best Fitness -14
Generation 3: Best Fitness -13
Generation 4: Best Fitness -13
Generation 5: Best Fitness -13
Generation 6: Best Fitness -13
Generation 7: Best Fitness -13
Generation 8: Best Fitness -13
Generation 9: Best Fitness -13
Generation 10: Best Fitness -13
Generation 11: Best Fitness -13
Generation 12: Best Fitness -13
Generation 13: Best Fitness -13
Generation 14: Best Fitness -9
Generation 15: Best Fitness -9
Generation 16: Best Fitness -9
Generation 17: Best Fitness -9
Generation 18: Best Fitness -9
Generation 19: Best Fitness -4
Generation 20: Best Fitness -4
Generation 21: Best Fitness -4
Generation 22: Best Fitness -4
Generation 23: Best Fitness -4
Generation 24: Best Fitness -4
Generation 25: Best Fitness -4
Generation 26: Best Fitness -4
Generation 27: Best Fitness -4
Generation 28: Best Fitness -4
Generation 29: Best Fitness -4
Generation 30: Best Fitness -4
Generation 31: Best Fitness -4
Gene

In [426]:
def generate_daily_timetables(detailed_chromosome):
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    timetables = {}
    for day in days:
        day_schedule = [gene for gene in detailed_chromosome if gene['First Lecture Day'] == day or gene['Second Lecture Day'] == day]
        schedule = []
        for gene in day_schedule:
            # Handle first lecture
            if gene['First Lecture Day'] == day:
                schedule.append({
                    'Time Slot': convert_to_twelve_hour(gene['First Lecture Timeslot'].replace(",", "-")),
                    'Course': gene['Course'],
                    'Type': gene['Type'],
                    'Section': gene['Section'],
                    'Room': gene['First Lecture Room'],
                    'Professor': gene['Professor']
                })
            # Handle second lecture
            if 'Second Lecture Day' in gene and gene['Second Lecture Day'] == day:
                schedule.append({
                    'Time Slot': convert_to_twelve_hour(gene['Second Lecture Timeslot'].replace(",", "-")),
                    'Course': gene['Course'],
                    'Type': gene['Type'],
                    'Section': gene['Section'],
                    'Room': gene['Second Lecture Room'],
                    'Professor': gene['Professor']
                })

        # Convert list of schedules to DataFrame and sort by 'Time Slot'
        if schedule:
            df = pd.DataFrame(schedule)
            df['Sort Key'] = df['Time Slot'].apply(lambda x: get_sort_key(x))
            df.sort_values(by='Sort Key', inplace=True)
            df.drop('Sort Key', axis=1, inplace=True)
            timetables[day] = df
        else:
            timetables[day] = pd.DataFrame(columns=['Time Slot', 'Course', 'Type', 'Section', 'Room', 'Professor'])

    return timetables

def get_sort_key(time_slot):
    """ Calculate a sort key based on the time and AM/PM """
    time, period = time_slot.split(' to ')[0].split(' ')[0], time_slot.split(' ')[1]
    hour, minute = map(int, time.split(':'))
    # Adjust hour for PM times for correct sorting unless it's 12 PM which is noon
    if period == 'PM' and hour != 12:
        hour += 12
    elif period == 'AM' and hour == 12:
        hour = 0  # Midnight case
    return hour * 100 + minute

def convert_to_twelve_hour(time_str):
    """Converts time from 'HH.MM' format to 'HH:MM AM/PM' format."""
    start, end = time_str.split('-')
    start_hour, start_minute = map(int, start.split('.'))
    end_hour, end_minute = map(int, end.split('.'))
    
    start_am_pm = 'AM' if start_hour < 12 else 'PM'
    end_am_pm = 'AM' if end_hour < 12 else 'PM'
    
    # Convert 24-hour time to 12-hour format
    start_hour = start_hour if start_hour <= 12 else start_hour - 12
    end_hour = end_hour if end_hour <= 12 else end_hour - 12
    
    start_formatted = f"{start_hour}:{start_minute:02d} {start_am_pm}"
    end_formatted = f"{end_hour}:{end_minute:02d} {end_am_pm}"
    
    return f"{start_formatted} to {end_formatted}"


def print_timetables(timetables):
    # Iterate over each day's timetable in the dictionary
    for day, timetable in timetables.items():
        print(f"Timetable for {day}")
        table = PrettyTable()
        table.field_names = timetable.columns.tolist()
        for idx, row in timetable.iterrows():
            table.add_row(row)
        print(table)
        print("\n")  # Add a newline for better separation between days


def write_timetables_to_file(timetables, file_path):
    with open(file_path, "w") as file:
        # Iterate over each day's timetable in the dictionary
        for day, timetable in timetables.items():
            file.write(f"Timetable for {day}\n")
            table = PrettyTable()
            table.field_names = timetable.columns.tolist()
            for idx, row in timetable.iterrows():
                table.add_row(row)
            file.write(table.get_string())
            file.write("\n\n")  # Add two newlines for better separation between days


timetables = generate_daily_timetables(best_chromosome[0])

# Print all timetables
print_timetables(timetables)

store_path = "TimeTables.txt"

write_timetables_to_file(timetables,store_path)



Timetable for Monday
+----------------------+--------+--------+---------+------+--------------+
|      Time Slot       | Course |  Type  | Section | Room |  Professor   |
+----------------------+--------+--------+---------+------+--------------+
| 10:00 AM to 11:20 AM | AI Lab |  Lab   |    4    |  8   | Naveed Ahmed |
| 11:30 AM to 12:50 PM | AI Lab |  Lab   |    4    |  8   | Naveed Ahmed |
|  2:30 PM to 3:50 PM  |  CNET  | Theory |    5    |  6   | Amina Ashfaq |
+----------------------+--------+--------+---------+------+--------------+


Timetable for Tuesday
+----------------------+------------------+--------+---------+------+--------------+
|      Time Slot       |      Course      |  Type  | Section | Room |  Professor   |
+----------------------+------------------+--------+---------+------+--------------+
| 10:00 AM to 11:20 AM |        OS        | Theory |    1    |  4   | Kashif Munir |
| 11:30 AM to 12:50 PM |        OS        | Theory |    5    |  1   | Kashif Munir |
|  1: